In [1]:
# For Preprocessing
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from xgboost import plot_importance
from sklearn.metrics import confusion_matrix
import numpy as np
from collections import Counter
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
import os
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
%matplotlib inline
import seaborn as sns
import dask.dataframe as dd
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score,recall_score,precision_score
from dask.distributed import Client

In [2]:
X = pd.read_csv("application_train.csv")
df_test = pd.read_csv("application_test.csv")
X.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
X['CODE_GENDER'].replace('XNA',np.nan, inplace=True)
X['CODE_GENDER'].value_counts()

F    202448
M    105059
Name: CODE_GENDER, dtype: int64

In [4]:
X['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)

In [5]:
X['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

In [6]:
# AGGREGATION_RECIPIES = [
#     (['CODE_GENDER', 'NAME_EDUCATION_TYPE'], [('AMT_ANNUITY', 'max')                                             
                                              
#                                               ]),
#     (['CODE_GENDER', 'ORGANIZATION_TYPE'], [
#                                             ('AMT_INCOME_TOTAL', 'mean'),
#                                             ('DAYS_REGISTRATION', 'mean')
#                                             ]),
#     (['CODE_GENDER', 'REG_CITY_NOT_WORK_CITY'], [
#                                                  ('CNT_CHILDREN', 'mean')]),
 
#     (['NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE'], [('AMT_CREDIT', 'mean'),
#                                                   ('AMT_REQ_CREDIT_BUREAU_YEAR', 'mean'),
#                                                   ('APARTMENTS_AVG', 'mean'),
#                                                   ('BASEMENTAREA_AVG', 'mean'),
#                                                   ('NONLIVINGAREA_AVG', 'mean'),
#                                                   ('OWN_CAR_AGE', 'mean'),
#                                                   ('YEARS_BUILD_AVG', 'mean')]),
#     (['NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'REG_CITY_NOT_WORK_CITY'], [('ELEVATORS_AVG', 'mean')
#                                                                             ])
# ]

# groupby_aggregate_names = []
# for groupby_cols, specs in tqdm(AGGREGATION_RECIPIES):
#     group_object = X.groupby(groupby_cols)
#     for select, agg in tqdm(specs):
#         groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
#         X = X.merge(group_object[select]
#                               .agg(agg)
#                               .reset_index()
#                               .rename(index=str,
#                                       columns={select: groupby_aggregate_name})
#                               [groupby_cols + [groupby_aggregate_name]],
#                               on=groupby_cols,
#                               how='left')
#         groupby_aggregate_names.append(groupby_aggregate_name)

In [7]:
# X_agg = X[groupby_aggregate_names + ['TARGET']]
# X_agg_corr = abs(X_agg.corr())
# X_agg_corr.sort_values('TARGET', ascending=False)['TARGET']

In [8]:
# X=X.drop(['AMT_ANNUITY','OWN_CAR_AGE','AMT_INCOME_TOTAL','DAYS_REGISTRATION','CNT_CHILDREN','AMT_CREDIT',
#           'AMT_REQ_CREDIT_BUREAU_YEAR','APARTMENTS_AVG','BASEMENTAREA_AVG','NONLIVINGAREA_AVG',
#           'YEARS_BUILD_AVG','ELEVATORS_AVG'
#          ],axis=1)
# #Dropped 12 columns and replaced them with their corresponding aggregated columns

In [9]:

le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in X:
    if X[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(X[col].unique())) <= 2:
            # Train on the training data
            le.fit(X[col])
            # Transform both training and testing data
            X[col] = le.transform(X[col])
           # df_test[col] = le.transform(df_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)


3 columns were label encoded.


In [10]:
# one-hot encoding of categorical variables
X = pd.get_dummies(X)
#df_test = pd.get_dummies(df_test)
list1=X.columns

print('Training Features shape: ', X.shape)
#print('Testing Features shape: ', df_test.shape)
print(list1)

Training Features shape:  (307511, 242)
Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       ...
       'HOUSETYPE_MODE_terraced house', 'WALLSMATERIAL_MODE_Block',
       'WALLSMATERIAL_MODE_Mixed', 'WALLSMATERIAL_MODE_Monolithic',
       'WALLSMATERIAL_MODE_Others', 'WALLSMATERIAL_MODE_Panel',
       'WALLSMATERIAL_MODE_Stone, brick', 'WALLSMATERIAL_MODE_Wooden',
       'EMERGENCYSTATE_MODE_No', 'EMERGENCYSTATE_MODE_Yes'],
      dtype='object', length=242)


In [11]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [12]:
# Missing values statistics
missing_values = missing_values_table(X)
mis_col=list(missing_values.index)

Your selected dataframe has 242 columns.
There are 62 columns that have missing values.


In [13]:
# #Have commented the dropping (not dropping any columns for now) Replace missing with -999
# Mis=X[mis_col][:]
# X_mis=(Mis.isnull().sum() / Mis.shape[0] * 100.00)
# df_mis=pd.DataFrame(np.c_[X_mis.index,X_mis],columns=['Colname','miss_p'])
# drop_col=df_mis[df_mis['miss_p']>65]['Colname'].values
# print('number of columns before dropping->',len(X.columns))
# #X=X.drop(drop_col,axis=1)
# print('number of columns after dropping->',len(X.columns))
X.replace(np.nan,-999, inplace=True)
#X.astype('float32',inplace=True)
#print(newX[].describe())
y=X['TARGET']
X=X.drop('TARGET',axis=1)

In [14]:
# iterate below functions only

In [15]:
def split_train_test(X , y):
    return(train_test_split( X , y , test_size=0.2, random_state=42))

X_train, X_test, y_train, y_test =split_train_test(X,y)

In [16]:
def feature_eng(X_train,y_train,X_test,y_test):
    train_col=list(X_train.columns)
    test_col=list(X_test.columns)    
    clf = ExtraTreesClassifier(n_estimators=50)
    clf = clf.fit(X_train, y_train)
    model = SelectFromModel(clf, prefit=True)
    new_trainX = model.transform(X_train)
    n=np.shape(new_trainX)[1]
    importances = clf.feature_importances_
    indices = np.argsort(importances)[::-1]
    clist=[]
    for f in range(n):     
        clist.append(indices[f])
    new_train_col=[train_col[i] for i in clist]
    new_test_col=[test_col[i] for i in clist]
    X_train=pd.DataFrame(X_train)
    X_test=pd.DataFrame(X_test)
    y_train=pd.DataFrame(y_train)
    y_test=pd.DataFrame(y_test)
    df_new_train=pd.DataFrame(X_train.loc[:,new_train_col],columns=new_train_col)            
    df_new_test=pd.DataFrame(X_test.loc[:,new_test_col],columns=new_test_col)
    print("Improved number of features->",len(clist))    
    return(df_new_train,df_new_test)

df_new_train,df_new_test=feature_eng(X_train, y_train, X_test, y_test)

Improved number of features-> 83


In [17]:
def run_model_eval_metric(modelname,modelobj,X_train,Y_train,X_test,Y_test):
    # In order to let the models run faster we use dask dataframe
    
    modelobj.fit(X_train,Y_train)
    ypred_train=modelobj.predict(X_train)
    ypred_test=modelobj.predict(X_test)
    y_train_proba = modelobj.predict_proba(X_train)[:,1]
    y_test_proba = modelobj.predict_proba(X_test)[:,1]
    print('\n')
    print(modelname,"model:")
    confmat=confusion_matrix(Y_train,ypred_train)
    print("Train Confusion Matrix:")
    print(confmat)
    confmat=confusion_matrix(Y_test,ypred_test)
    print("Test Confusion Matrix:")
    print(confmat)
    metric=pd.DataFrame([[0]*5]*2,columns=['PRECISION','RECALL','F1_SCORE','ROC_AUC_SCORE','ACCURACY'],index=['Train','Test'],dtype='float32')
    metric['PRECISION']['Train']=round(precision_score(Y_train, ypred_train),3)
    metric['RECALL']['Train']=recall_score(Y_train, ypred_train)
    metric['F1_SCORE']['Train']=round(f1_score(Y_train, ypred_train),3)
    metric['ROC_AUC_SCORE']['Train']=roc_auc_score(Y_train,y_train_proba)
    metric['ACCURACY']['Train']=accuracy_score(Y_train,ypred_train)
    metric['PRECISION']['Test']=precision_score(Y_test, ypred_test)
    metric['RECALL']['Test']=recall_score(Y_test, ypred_test)
    metric['F1_SCORE']['Test']=f1_score(Y_test, ypred_test)
    metric['ROC_AUC_SCORE']['Test']=roc_auc_score(Y_test,y_test_proba)
    metric['ACCURACY']['Test']=accuracy_score(Y_test,ypred_test)
    print(metric)
   
    

In [18]:
def run_all_models(df_new_train,y_train,df_new_test,y_test):
    # K Nearest Neighbor
    knearest = KNeighborsClassifier()
    run_model_eval_metric('K-Nearest Neighbor ',knearest,df_new_train,y_train,df_new_test,y_test)
    
#     knearest = KNeighborsClassifier()
#     run_model_eval_metric('K-Nearest Neighbor 3',knearest,df_new_train,y_train,df_new_test,y_test)

    # Random Forest
    randfor = RandomForestClassifier()
    run_model_eval_metric('Random Forest',randfor,df_new_train,y_train,df_new_test,y_test)

    # Decision Trees
    dectree = DecisionTreeClassifier()
    run_model_eval_metric('Decision Tree',dectree,df_new_train,y_train,df_new_test,y_test)

    # Logistic Regression
    logreg = LogisticRegression()
    run_model_eval_metric('Logistic Regression',logreg,df_new_train,y_train,df_new_test,y_test)

run_all_models(df_new_train,y_train,df_new_test,y_test)




K-Nearest Neighbor 1 model:
Train Confusion Matrix:
[[225409    723]
 [ 18724   1152]]
Test Confusion Matrix:
[[56177   377]
 [ 4890    59]]
       PRECISION    RECALL  F1_SCORE  ROC_AUC_SCORE  ACCURACY
Train   0.614000  0.057959  0.106000       0.887909  0.920950
Test    0.135321  0.011922  0.021913       0.541189  0.914362


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)




Random Forest model:
Train Confusion Matrix:
[[226129      3]
 [  3626  16250]]
Test Confusion Matrix:
[[56457    97]
 [ 4916    33]]
       PRECISION    RECALL  F1_SCORE  ROC_AUC_SCORE  ACCURACY
Train   1.000000  0.817569  0.900000       0.999822  0.985248
Test    0.253846  0.006668  0.012995       0.627320  0.918492


Decision Tree model:
Train Confusion Matrix:
[[226132      0]
 [     0  19876]]
Test Confusion Matrix:
[[51488  5066]
 [ 4091   858]]
       PRECISION    RECALL  F1_SCORE  ROC_AUC_SCORE  ACCURACY
Train   1.000000  1.000000  1.000000       1.000000  1.000000
Test    0.144835  0.173368  0.157822       0.541895  0.851113


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)




Logistic Regression model:
Train Confusion Matrix:
[[226126      6]
 [ 19875      1]]
Test Confusion Matrix:
[[56552     2]
 [ 4949     0]]
       PRECISION   RECALL  F1_SCORE  ROC_AUC_SCORE  ACCURACY
Train      0.143  0.00005       0.0       0.635367  0.919186
Test       0.000  0.00000       0.0       0.640536  0.919500


In [19]:
# #Run Base Models
# X_train, X_test, y_train, y_test =split_train_test(X,y)
# df_new_train,df_new_test=feature_eng(X_train, y_train, X_test, y_test)
# df_new_train=dd.from_pandas(df_new_train,npartitions=3)
# df_new_test=dd.from_pandas(df_new_test,npartitions=3)
# y_train=dd.from_pandas(y_train,npartitions=3)
# y_test=dd.from_pandas(y_test,npartitions=3)
# run_all_models(df_new_train,y_train,df_new_test,y_test)

In [ ]:
# #Handling Imbalance problem
# X_tr, X_test, y_tr, y_test =split_train_test(X,y)
# '''
# Use SMOTE to handle imbalance problem
# '''
# # Class0 size is 10 times that of Class1 in train set
# print('Y train before resampling',Counter(y_tr))
# Imb_sm = SMOTE(random_state=42)
# X_train, y_train = Imb_sm.fit_resample(X_tr, y_tr)
# print('Y train after resampling',Counter(y_train))
# X_train=pd.DataFrame(X_train)
# y_train=pd.DataFrame(y_train)
# df_new_train,df_new_test=feature_eng(X_train, y_train, X_test, y_test)
# df_new_train=dd.from_pandas(df_new_train,npartitions=3)
# df_new_test=dd.from_pandas(df_new_test,npartitions=3)
# y_train=dd.from_pandas(y_train,npartitions=3)
# y_test=dd.from_pandas(y_test,npartitions=3)
# run_all_models(df_new_train,y_train,df_new_test,y_test)